# Convex room RIR calculation

{download}`Download this page as a notebook 🗎 </guides/convex_room.ipynb>`.


Let's calculate the room impulse response (RIR) for a convex room using torchRIR. First let's define a shoebox room as a convex room, given it's vertices.

In [ ]:
from torchrir.geometry import ConvexRoom
import torch

torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float64)

x_size = 4.5000
y_size = 2.1500
z_size = 3.3000
points = torch.tensor(
    [
        [-x_size, -y_size, -z_size],
        [-x_size, -y_size, z_size],
        [-x_size, y_size, -z_size],
        [-x_size, y_size, z_size],
        [x_size, -y_size, -z_size],
        [x_size, -y_size, z_size],
        [x_size, y_size, -z_size],
        [x_size, y_size, z_size],
    ],
)

shoebox_room = ConvexRoom(points.T, 0.9)

We can simply visualize the room using matplotlib by calling `.plot()`.

In [ ]:
shoebox_room.plot(alpha=0.35)

To compute the RIR, we need to define a sound source. which can be done by specifying the source position in the room.


In [ ]:
from torchrir.source import Source


source = Source(position=torch.tensor([0.8, 0.8, 0.8]), intensity=2.0)

Finally the impulse response can be computed by calling the `.computer_rir()` method on the room object, passing the receiver position, the sources, and the sampling frequency, along with other parameters.


In [ ]:
from torchrir.geometry import ImpulseResponseStrategies

fs = 28000.0  # Hz
rir = shoebox_room.compute_rir(
    p=torch.tensor([0.2, 0.2, 0.2]),  # receiver position
    s=source,
    k=7,
    fs=fs,
    impulse_response_fn=ImpulseResponseStrategies.sinc,
)

We can visualize the RIR in the time domain and in the frequency domain using matplotlib, as below.

In [ ]:
import matplotlib.pyplot as plt
import scipy


rir = [_.cpu() for _ in rir]  # move to CPU for plotting
fig = plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(rir[1], rir[0])
plt.xlim([0, 1])
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.subplot(1, 2, 2)
H = abs(scipy.fft.fft(rir[0].numpy()))
f = scipy.fft.fftfreq(len(H), 1 / fs)
plt.semilogx(f[: len(f) // 2], H[: len(f) // 2])
plt.xlim([1, fs / 2])
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.tight_layout()

The impulse response can also be heard below.

In [ ]:
import IPython
import wave

with wave.open("_output.wav", "w") as wf:
    wf.setnchannels(1)  # Mono
    wf.setsampwidth(2)  # 2 bytes per sample (16-bit PCM)
    wf.setframerate(1 / (rir[1][1] - rir[1][0]).numpy())
    wf.writeframes((rir[0].numpy() * 32767).astype("int16").tobytes())

IPython.display.Audio("_output.wav")